In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### 도구 호출 (Tool Calling)
> 웹 검색 기능을 AI모델에 통합해주는 웹 검색 API

- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음

랭체인 내장 도구 : Tavily

In [ ]:
# poetry add langchain-tavily
# 최신 langchain 0.3.25 최신버전 기준  
# langchain-tavily 포함된 클래스를 사용

from langchain_tavily.tavily_search import TavilySearch

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearch(max_results=3)

# 웹 검색 실행
search_results = web_search.invoke(query)
print(type(search_results))
#pprint(search_results)

# 검색 결과 출력
for result in search_results['results']:
    print(type(result))
    pprint(result)  
    print("-" * 100)  

<class 'dict'>
<class 'dict'>
{'content': 'Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 '
            '고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, '
            '스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 '
            '와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 '
            '저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 '
            '‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 '
            '방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 '
            '레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. '
            'Tags: 마시자매거진 소고기와인 스테이크와인 와인추천 와인페어링 Bora Kim Previous Article '
            'Next Article Your email address will not be published. 대표자 : 방문송 '
            '사업자등록번호 : 325-87-00031 발행인 / 편집인 : 방문송',
 'raw_content': None,
 'score': 0.8668671,
 'title': '스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진',
 'url': 'https://mashija.com/%E

In [5]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_tavily.tavily_search.TavilySearch'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. It not '
 'only retrieves URLs and snippets, but offers advanced search depths, domain '
 'management, time range filters, and image search, this tool delivers '
 'real-time, accurate, and citation-backed results.Input should be a search '
 'query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for [TavilySearch]',
 'properties': {'exclude_domains': {'anyOf': [{'items': {'type': 'string'},
                                               'type': 'array'},
        

### 도구 호출
- bind_tools로 LLM에 직접 바인딩

In [6]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

<class 'langchain_core.runnables.base.RunnableBinding'>


In [9]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)
print(type(ai_msg))

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 770, 'total_tokens': 782, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmiVt0wervDadmsd0DFwDQn75ZepP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8d033ae6-4d73-49f2-bbb1-aae1e16ff137-0', usage_metadata={'input_tokens': 770, 'output_tokens': 12, 'total_tokens': 782, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
---------

In [11]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
#pprint(dir(ai_msg))

In [12]:
pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_M57MR6KSLM4ijdOG9t3wQ6KP', 'function': {'arguments': '{"query":"스테이크에 어울리는 와인 추천","search_depth":"advanced"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 779, 'total_tokens': 809, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmiYptNXRq2lZCDYEzS5K6Da0ECns', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--57ed90ca-0ce5-4f64-88c0-87ec8c05b58c-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '스테이크에 어울리는 와인 추천', 'search_depth': 'advanced'}, 'id': 'call_M57MR6KSLM4ijdOG9t3wQ6KP', 'type': 'tool_call'}], usage_metadata={'input_t

### 도구 실행하기

#### 1. 직접 도구 호출 처리한 결과로 ToolMessage 생성
- 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
- 'args' 속성을 사용하여 TavilySearch 객체의 invoke() 호출하여 결과를 얻는다.

In [13]:
tool_call = ai_msg.tool_calls[0]

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search 호출 결과:
----------------------------------------------------------------------------------------------------
{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '스테이크에 어울리는 와인 추천',
 'response_time': 1.7,
 'results': [{'content': '• 카베르네 소비뇽(Cabernet Sauvignon)  \n'
                         '• 말벡(Malbec)  \n'
                         '• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n'
                         '• 시라/쉬라즈(Syrah / Shiraz)  \n'
                         '• 산지오베제(Sangiovese)\n'
                         '\n'
                         '육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 '
                         '즐거움일 것이다.\n'
                         '\n'
                         '와인과 음식 페어링에서 새로운 시도를 하는 것은 항상 재미있지만, 특별한 스테이크 저녁 식사를 '
                         '준비할 때 고려해야 할 몇 가지 스타일과 주의사항이 있다.\n'
                         '\n'
                         '**<스테이크에 곁들이는 레드 와인\\>**\n'
                         '\n'
                         '이 포도 품종을 세계 와인 무대에

#### (추천) 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성
- tool_call 변수는 AIMessage 객체 포함된 tool 을 호출한 결과 tool_calls의 첫번째 dict 객체
- TavilySearch 객체를 invoke(tool_call) 하면 결과객체의 타입이 ToolMessage 이다.
- 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
- 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [14]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")

<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: {"query": "스테이크에 어울리는 와인 추천", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "title": "스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진", "content": "• 카베르네 소비뇽(Cabernet Sauvignon)  \n• 말벡(Malbec)  \n• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  \n• 시라/쉬라즈(Syrah / Shiraz)  \n• 산지오베제(Sangiovese)\n\n육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.\n\n와인과 음식 페어링에서 새로운 시도를 하는 것은 항상 재미있지만, 특별한 스테이크 저녁 식사를 준비할 때 고려해야 할 몇 가지 스타일과 주의사항이 있다.\n\n**<스테이크에 곁들이는 레드 와인\\>**\n\n이 포도 품종을 세계 와인 무대에 재등장시키고 고품질 쇠고기에 대한 국가의 명성을 가진 아르헨티나 덕분에, 말벡 레드 와인은 스테이크와 함께 고전적인 매칭이 되었다.\n\n말벡의 풍부한 짙은 과일의 특징과 자연스러운 타닌은 일반적으로 좋은 스테이크와 잘 어울린다고 여겨지지만, 일부 전문가들은 더 신선한 스타일을 찾는 것을 제안한다. [...] 카베르네 소비뇽(C